In [1]:
import os
import gc
import sys
import random
import datetime
import importlib
import itertools
import numpy as np
from scipy import spatial
import scipy.sparse as sparse
import scipy.stats as stats
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns

from mayavi import mlab

# os.chdir('/media/sina/Windows1/Users/smansourlako/Documents/Reserach/Codes/fMRI')
os.chdir('/home/sina/Documents/Research/Codes/fMRI')

# import constants as cs
import myconstants as cs
# import importlib.util
# spec = importlib.util.spec_from_file_location('cs', '/home/sina/Documents/Research/Codes/fMRI/constants.py')
# cs = importlib.util.module_from_spec(spec)
# sys.modules['cs'] = cs
# spec.loader.exec_module(cs)
import utils
import niutils
import hcp


---

Let's first load all the files needed:

In [2]:
# load the atlas file

atlas_file = "/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000243_2_0/dMRI/dMRI/atlases/combinations/native.dMRI_space.aparc.a2009s+Tian_Subcortex_S1_3T.nii.gz"
# atlas_file = "/home/sina/Documents/Research/Codes/UKB-connectomics/data/temporary/subjects/1000243_2_0/tractography/atlases/native.dMRI_space.aparc.a2009s.nii.gz"

atlas=nib.load(atlas_file)

# load the tractography endpoint information

endpoint_file = "/home/sina/Documents/Research/Codes/UKB-connectomics/data/temporary/subjects/1000243_2_0/tractography/endpoints/tracks_10M_endpoints.npy"

endpoints = np.load(endpoint_file)


---

Now we need to extract appropriate voxel coordinates from the atlas:

In [3]:
node_indices = np.arange(np.prod(atlas.shape)).reshape(atlas.shape)

ind_i, ind_j, ind_k = np.meshgrid(
    np.arange(atlas.shape[0]),
    np.arange(atlas.shape[1]),
    np.arange(atlas.shape[2]), indexing='ij',
)

node_ijk = np.array([ind_i.reshape(-1), ind_j.reshape(-1), ind_k.reshape(-1),]).T

node_xyz = nib.affines.apply_affine(atlas.affine, node_ijk)


In [4]:
# only select voxels with a label greater than zero
selection_mask = (atlas.get_fdata() > 0)

selection_indices = node_ijk[selection_mask.reshape(-1), :]

selection_xyz = node_xyz[selection_mask.reshape(-1), :]

selection_labels = atlas.get_fdata()[selection_mask].astype(int)


In [5]:
%%time
# build a kdtree for spatial queries
kdtree = spatial.cKDTree(selection_xyz)

# get the list of endpoints
starts = endpoints[:,0,:]
ends = endpoints[:,-1,:]

# query for closest coordinate from selection
start_dists, start_indices = kdtree.query(starts)
end_dists, end_indices = kdtree.query(ends)

# mask points that are further than the search radius from all selection coordinates
search_radius = 4
distance_mask = (start_dists < search_radius) & (end_dists < search_radius)


CPU times: user 6.75 s, sys: 281 ms, total: 7.03 s
Wall time: 7.03 s


In [6]:
%%time
# now generate a connectivity matrix

# only keep valid endpoints according to the search radius
valid_start_indices = start_indices[distance_mask]
valid_end_indices = end_indices[distance_mask]

# number of regions/nodes
node_count = selection_labels.max()

# generate connectivity matrix
adj = np.zeros((node_count, node_count), dtype=np.float32)
np.add.at(adj, (selection_labels[valid_start_indices] - 1, selection_labels[valid_end_indices] - 1), 1)
adj = adj + adj.T
adj[np.diag_indices_from(adj)] /= 2


CPU times: user 194 ms, sys: 12.2 ms, total: 206 ms
Wall time: 205 ms


In [19]:
adj[np.diag_indices_from(adj)] /= 2


---

Let's also compare with mrtix generate adjacency:

In [8]:
mradj = np.loadtxt("/home/sina/Documents/Research/Codes/UKB-connectomics/data/temporary/subjects/1000243_2_0/tractography/connectomes/aparc.a2009s+Tian_Subcortex_S1_3T/connectome_streamline_count_10M.csv", delimiter=',')

In [9]:
mradj

array([[7.370e+02, 1.100e+01, 0.000e+00, ..., 4.400e+01, 7.600e+02,
        4.280e+02],
       [1.100e+01, 3.048e+03, 0.000e+00, ..., 1.000e+00, 1.060e+02,
        2.000e+00],
       [0.000e+00, 0.000e+00, 5.647e+03, ..., 2.990e+02, 4.890e+02,
        1.170e+02],
       ...,
       [4.400e+01, 1.000e+00, 2.990e+02, ..., 6.200e+01, 1.404e+03,
        6.500e+01],
       [7.600e+02, 1.060e+02, 4.890e+02, ..., 1.404e+03, 5.933e+03,
        2.620e+02],
       [4.280e+02, 2.000e+00, 1.170e+02, ..., 6.500e+01, 2.620e+02,
        2.864e+03]])

In [14]:
np.sum(adj!=mradj)

2407

In [15]:
np.sum(adj==mradj)

24489

In [16]:
np.corrcoef(adj.reshape(-1), mradj.reshape(-1))

array([[1.        , 0.99999966],
       [0.99999966, 1.        ]])

---

Store output as csv:

In [ ]:
np.savetxt('tmp.csv', adj, delimiter=',')